<a href="https://colab.research.google.com/github/lawrennd/spectral/blob/main/examples/03_image_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Segmentation: Effect of Sigma

## Neil D. Lawrence

## 2026-02-08

This notebook demonstrates spectral clustering for image segmentation on the shapes image from:

> **Automatic Determination of the Number of Clusters using Spectral Algorithms**  
> Sanguinetti, G., Laidler, J., Lawrence, N.D. (2005)  
> IEEE Workshop on Machine Learning for Signal Processing (NNSP 2005)

## The Challenge

Image segmentation: partition an image into regions based on both **spatial proximity** and **intensity similarity**.

Key insight: We can control segmentation granularity by adjusting $\sigma$:
- **Smaller sigma**: Finer segmentation (more clusters)
- **Larger sigma**: Coarser segmentation (fewer clusters)

## What we'll demonstrate:

1. Load the shapes image
2. Create feature vectors: (x, y, intensity)
3. Cluster with two different sigma values
4. Compare coarse vs. fine segmentation
5. Reproduce paper Figures 5b and 5c

In [ ]:
# Install spectral-cluster package if needed
import sys
from pathlib import Path

try:
    import spectral
    print(f"✓ spectral package already installed (version {spectral.__version__})")
except ImportError:
    print("📦 Installing spectral-cluster package...")

    here = Path.cwd().resolve()
    parent = here.parent

    if (parent / "pyproject.toml").exists() and (parent / "spectral").is_dir():
        print(f"  → Installing from local directory: {parent}")
        import subprocess
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-e", str(parent)],
            stdout=subprocess.DEVNULL
        )
    else:
        print("  → Installing from GitHub...")
        import subprocess
        subprocess.check_call([
            sys.executable, "-m", "pip", "install",
            "git+https://github.com/lawrennd/spectral.git"
        ])

    import spectral
    print(f"✓ spectral package installed successfully (version {spectral.__version__})")

In [ ]:
# Import required packages
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from spectral import SpectralCluster

# Set random seed
np.random.seed(1)

# Configure matplotlib
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['font.size'] = 11

print('✓ All packages loaded successfully')

## 1. Load and Prepare the Image

For image segmentation, we create a feature vector for each pixel:
- **Spatial features**: (x, y) coordinates
- **Intensity feature**: grayscale value

The intensity is weighted to give it equal importance to spatial features.

In [ ]:
def ensure_data_files():
    """Download data files if they don't exist locally."""
    from pathlib import Path
    import urllib.request

    data_dir = Path('data')
    data_dir.mkdir(exist_ok=True)

    base_url = 'https://raw.githubusercontent.com/lawrennd/spectral/main/examples/data/'
    files = ['shapes.bmp']

    for filename in files:
        filepath = data_dir / filename
        if not filepath.exists():
            print(f'📥 Downloading {filename}...')
            urllib.request.urlretrieve(base_url + filename, filepath)
            print(f'  ✓ Downloaded to {filepath}')

# Ensure data files are available
ensure_data_files()

def prepare_image_features(filename, downsample=1):
    """Load image and create (x, y, intensity) feature vectors."""
    # Load and convert to grayscale
    img = Image.open(f'data/{filename}')
    img_array = np.array(img)

    # Convert RGB to grayscale if needed
    if len(img_array.shape) == 3:
        img_gray = img_array.mean(axis=2)
    else:
        img_gray = img_array.astype(float)

    # Normalize to [0, 1]
    img_norm = img_gray / img_gray.max()

    # Downsample if requested
    if downsample > 1:
        img_norm = img_norm[::downsample, ::downsample]

    # Get dimensions
    numrows, numcols = img_norm.shape

    # Create coordinate grids
    y_coords, x_coords = np.mgrid[0:numrows, 0:numcols]

    # Flatten into vectors
    x_flat = x_coords.ravel()
    y_flat = y_coords.ravel()
    intensity_flat = img_norm.ravel()

    # Weight intensity to match spatial scale
    # This gives intensity equal importance to spatial coordinates
    intensity_weighted = intensity_flat * (numrows + numcols)

    # Combine into feature matrix: (x, y, weighted_intensity)
    X = np.column_stack([x_flat, y_flat, intensity_weighted])

    return X, img_norm, (numrows, numcols)

# Load the shapes image
X, img_norm, (numrows, numcols) = prepare_image_features('shapes.bmp', downsample=1)

print(f"Image size: {numrows} × {numcols}")
print(f"Total pixels: {X.shape[0]}")
print(f"Feature dimensions: {X.shape[1]} (x, y, intensity)")

In [ ]:
# Visualize the original image
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.imshow(img_norm, cmap='gray')
ax.set_title('Original Shapes Image (Paper Figure 5a)', fontsize=13, fontweight='bold')
ax.axis('off')
plt.tight_layout()
plt.show()

print("\nChallenge: How many distinct regions are there?")
print("Let's see what the algorithm finds with different sigma values!")

## 2. Coarse Segmentation (sigma = 0.707)

First, we use a smaller sigma for coarser segmentation.
- **MATLAB**: sigma²=1 → **Python**: $\sigma = \sqrt{1/2} \approx 0.707$

This should produce fewer, larger regions.

In [ ]:
# Cluster with sigma = 0.707 (coarse segmentation)
clf_coarse = SpectralCluster(sigma=0.707, random_state=1)
clf_coarse.fit(X)

print(f"\n{'='*60}")
print(f"COARSE SEGMENTATION (sigma = 0.707)")
print(f"{'='*60}")
print(f"Number of clusters detected: {clf_coarse.n_clusters_}")
print(f"Algorithm used {clf_coarse.eigenvectors_.shape[1]} eigenvectors")
print(f"{'='*60}\n")

In [ ]:
# Reshape labels back to image shape
labels_coarse_img = clf_coarse.labels_.reshape(numrows, numcols)

# Visualize coarse segmentation
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Original image
axes[0].imshow(img_norm, cmap='gray')
axes[0].set_title('Original Image', fontsize=13, fontweight='bold')
axes[0].axis('off')

# Segmentation result
im = axes[1].imshow(labels_coarse_img, cmap='tab20', interpolation='nearest')
axes[1].set_title(f'Coarse Segmentation: {clf_coarse.n_clusters_} clusters (sigma=0.707)\n(Paper Figure 5b)',
                  fontsize=13, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im, ax=axes[1], label='Cluster', fraction=0.046)

plt.tight_layout()
plt.show()

print(f"Paper Figure 5b reproduced: {clf_coarse.n_clusters_} large regions identified.")

## 3. Fine Segmentation (sigma = 1.0)

Now we use a larger sigma for finer segmentation.
- **MATLAB**: sigma²=2 → **Python**: $\sigma = \sqrt{2/2} = 1.0$

This should produce more, smaller regions - capturing finer details.

In [ ]:
# Cluster with sigma = 1.0 (fine segmentation)
clf_fine = SpectralCluster(sigma=1.0, random_state=1)
clf_fine.fit(X)

print(f"\n{'='*60}")
print(f"FINE SEGMENTATION (sigma = 1.0)")
print(f"{'='*60}")
print(f"Number of clusters detected: {clf_fine.n_clusters_}")
print(f"Algorithm used {clf_fine.eigenvectors_.shape[1]} eigenvectors")
print(f"{'='*60}\n")

In [ ]:
# Reshape labels back to image shape
labels_fine_img = clf_fine.labels_.reshape(numrows, numcols)

# Visualize fine segmentation
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Original image
axes[0].imshow(img_norm, cmap='gray')
axes[0].set_title('Original Image', fontsize=13, fontweight='bold')
axes[0].axis('off')

# Segmentation result
im = axes[1].imshow(labels_fine_img, cmap='tab20', interpolation='nearest')
axes[1].set_title(f'Fine Segmentation: {clf_fine.n_clusters_} clusters (sigma=1.0)\n(Paper Figure 5c)',
                  fontsize=13, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im, ax=axes[1], label='Cluster', fraction=0.046)

plt.tight_layout()
plt.show()

print(f"Paper Figure 5c reproduced: {clf_fine.n_clusters_} detailed regions identified.")

## 4. Side-by-Side Comparison

Let's compare both segmentations to see the effect of sigma.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Original
axes[0].imshow(img_norm, cmap='gray')
axes[0].set_title('Original Image', fontsize=13, fontweight='bold')
axes[0].axis('off')

# Coarse
im1 = axes[1].imshow(labels_coarse_img, cmap='tab20', interpolation='nearest')
axes[1].set_title(f'Coarse: {clf_coarse.n_clusters_} clusters\n(sigma=0.707)',
                  fontsize=13, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im1, ax=axes[1], label='Cluster', fraction=0.046)

# Fine
im2 = axes[2].imshow(labels_fine_img, cmap='tab20', interpolation='nearest')
axes[2].set_title(f'Fine: {clf_fine.n_clusters_} clusters\n(sigma=1.0)',
                  fontsize=13, fontweight='bold')
axes[2].axis('off')
plt.colorbar(im2, ax=axes[2], label='Cluster', fraction=0.046)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("COMPARISON SUMMARY")
print("="*60)
print(f"Coarse (sigma=0.707): {clf_coarse.n_clusters_} regions")
print(f"Fine (sigma=1.0):      {clf_fine.n_clusters_} regions")
print(f"\nLarger sigma → More clusters (finer detail)")
print(f"Smaller sigma → Fewer clusters (coarser grouping)")
print("="*60)

## 5. Understanding the Feature Space

Why combine spatial and intensity features?

- **Spatial only** (x, y): Would segment by location, ignoring intensity
- **Intensity only**: Would group similar colors regardless of location
- **Combined** (x, y, intensity): Groups nearby pixels with similar intensities

The weighting factor $(n_{rows} + n_{cols})$ ensures intensity has equal importance to spatial coordinates.

In [ ]:
# Visualize feature distribution for a few clusters (coarse segmentation)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Show clusters in eigenspace
eigenvecs = clf_coarse.eigenvectors_
scatter = axes[0].scatter(eigenvecs[:, 0], eigenvecs[:, 1],
                          c=clf_coarse.labels_, cmap='tab20', s=1, alpha=0.3)
axes[0].scatter(clf_coarse.centers_[:, 0], clf_coarse.centers_[:, 1],
                c='red', s=100, marker='d', edgecolors='k', linewidths=2,
                label='Centers')
axes[0].scatter([0], [0], c='black', s=100, marker='X',
                edgecolors='red', linewidths=2, label='Origin')
axes[0].set_xlabel('Eigenvector 1', fontsize=12)
axes[0].set_ylabel('Eigenvector 2', fontsize=12)
axes[0].set_title('Eigenspace (Coarse)', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_aspect('equal')

# Show clusters in eigenspace for fine segmentation
eigenvecs_fine = clf_fine.eigenvectors_
scatter2 = axes[1].scatter(eigenvecs_fine[:, 0], eigenvecs_fine[:, 1],
                           c=clf_fine.labels_, cmap='tab20', s=1, alpha=0.3)
axes[1].scatter(clf_fine.centers_[:, 0], clf_fine.centers_[:, 1],
                c='red', s=100, marker='d', edgecolors='k', linewidths=2,
                label='Centers')
axes[1].scatter([0], [0], c='black', s=100, marker='X',
                edgecolors='red', linewidths=2, label='Origin')
axes[1].set_xlabel('Eigenvector 1', fontsize=12)
axes[1].set_ylabel('Eigenvector 2', fontsize=12)
axes[1].set_title('Eigenspace (Fine)', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_aspect('equal')

plt.tight_layout()
plt.show()

print("\nEigenspace shows radial clustering structure.")
print("More eigenvectors needed for fine segmentation → more radial directions.")

## 6. Summary and Insights

### Results

Successfully demonstrated automatic image segmentation with two granularities:
- **Coarse** (sigma=0.707): Fewer, larger regions
- **Fine** (sigma=1.0): More, detailed regions

### Key Insights

1. **Sigma controls granularity**: Not just a "bandwidth" - it fundamentally changes what constitutes a cluster

2. **Feature engineering matters**:
   - Combining spatial + intensity creates meaningful clusters
   - Weighting ensures balanced importance

3. **Automatic detection**:
   - Don't need to specify number of segments
   - Algorithm finds natural groupings

4. **Eigenspace dimensionality**:
   - Finer segmentation requires more eigenvectors
   - Each eigenvector captures a different "direction" of variation

### Practical Guidance

**Choosing Sigma**:
- Start with $\sigma \approx$ (average distance between neighbors)
- Too small: Over-segmentation (every pixel its own cluster)
- Too large: Under-segmentation (everything in one cluster)
- Run with multiple values and inspect results

**When to use this approach**:
- Uncertain about number of segments
- Non-convex or irregular regions
- Want to explore at different scales

See notebook 5 for systematic parameter exploration.

## Conclusion

This notebook demonstrated:
- Image segmentation with spatial + intensity features
- Effect of sigma on segmentation granularity
- Reproduced paper Figures 5a, 5b, and 5c
- Automatic detection at multiple scales

## References

- Paper: Sanguinetti et al. (2005), Figures 5a, 5b, 5c
- MATLAB implementation: `matlab/demoShapes.m`, `matlab/demoShapes2.m`
- Python implementation: `spectral/cluster.py`